### DataLoading

In [1]:
from langchain.document_loaders import UnstructuredURLLoader
import os
import time
from langchain_together import Together
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
import pickle
from secret_key import Together_key
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [13]:
os.environ['TOGETHER_API_KEY'] = Together_key
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=128
)

In [3]:
loader = UnstructuredURLLoader(
    urls = [
        'https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-743-kr-choksey-11080811.html',
        'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html',
        'https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html'
    ]
)

In [4]:
data = loader.load()
len(data)

3

In [5]:
data[0].page_content[0:100]

'English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Pro'

In [6]:
data[0].metadata

{'source': 'https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-743-kr-choksey-11080811.html'}

### TextSplitting

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)
len(docs)

22

In [8]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-743-kr-choksey-11080811.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nNetwork 18\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_STOCKS_AS/MC_ENG_ROS_NWS_STK_AS_ATF_728\n\nMoneycontrol\n\nGo PRO NowPRO\n\nMoneycontrol PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nSubscription Products\n\nprofile\n\nAmbareesh Baliga\n\nprofile\n\nCK Narayan\n\nprofile\n\nT G

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore_mistral = FAISS.from_documents(docs, embedding_model)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9348\275808192.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Storing vector index create in local
file_path="sample_vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorstore_mistral, f)

In [11]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

## Retrieving

In [15]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())

query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

import logging

# See everything LangChain is doing
logging.basicConfig(level=logging.DEBUG)


chain({"question": query}, return_only_outputs=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9348\259271159.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"question": query}, return_only_outputs=True)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.together.xyz:443
DEBUG:urllib3.connectionpool:https://api.together.xyz:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.together.xyz:443
DEBUG:urllib3.connectionpool:https://api.together.xyz:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.together.xyz:443
DEBUG:urllib3.connectionpool:https://api.together.xyz:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.together.xyz:443
DEBUG:urllib3.connectionpool:https://api.together.xyz:443 "POST /v1/completions HTTP/1.1" 200 

{'answer': ' The price of Tiago iCNG is between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}